In [1]:
# from IPython.display import clear_output
# !pip install geopy
# !pip install geopandas
# !pip install basemap
# !pip install folium
# !pip install leafmap
# !pip install global_land_mask
# clear_output()

In [2]:
import openpyxl
from IPython.display import clear_output
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import leafmap.foliumap as leafmap
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import csv
import os
import pandas as pd
import folium
import random
from folium import plugins
import time
from global_land_mask import globe

In [3]:
def import_excel_data(file_path, sheet_name):
    # Load the workbook and select the desired sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook[sheet_name]

    # Get labels from row 2 (B2 to G2)
    labels = [sheet.cell(row=2, column=col).value for col in range(2, 7)]  # Columns B-G
    # print(labels)
    
    # Check if all labels are present
    if not all(labels):
        raise ValueError("One or more labels in row 2 are missing.")

    # Initialize a dictionary to store the data for each label
    data = {label: [] for label in labels}
    
    # Get data from rows starting at B3 (columns B-G)
    for row in sheet.iter_rows(min_row=3, min_col=2, max_col=7, values_only=True):
        for col_index, value in enumerate(row):
            if value:  # Skip empty cells
                data[labels[col_index]].append(value)
    return data

In [4]:
# Create four CSV files from the Excel Database for users without excel
def excel_to_csv(excel_file):
    # Load the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Iterate through each sheet and save it as a separate CSV file
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        csv_filename = f"{sheet_name}.csv"
        df.to_csv(csv_filename, index=False)
        print(f"Saved {csv_filename}")


In [5]:
excel_file = './Photonic Companies.xlsx'  # Replace with your actual file name
excel_to_csv(excel_file)

Saved Startup.csv
Saved Large Company.csv
Saved Foundries.csv
Saved Defense Contractors.csv


In [6]:
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Startup'  # Replace with the name of the sheet

try:
    data_SU = import_excel_data(file_path, sheet_name)
    print(data_SU)
except Exception as e:
    print(f"Error: {e}")
time.sleep(2)
clear_output()

In [7]:
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Large Company'  # Replace with the name of the sheet

try:
    data_LC = import_excel_data(file_path, sheet_name)
    print(data_LC)
except Exception as e:
    print(f"Error: {e}")
time.sleep(2)
clear_output()

In [8]:
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Foundries'  # Replace with the name of the sheet

try:
    data_F = import_excel_data(file_path, sheet_name)
    print(data_F)
except Exception as e:
    print(f"Error: {e}")
time.sleep(2)
clear_output()

In [9]:
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Defense Contractors'  # Replace with the name of the sheet

try:
    data_DC = import_excel_data(file_path, sheet_name)
    print(data_DC)
except Exception as e:
    print(f"Error: {e}")
time.sleep(2)
clear_output()

### If a company has multiple locations splits that into new rows and then duplicate the information to the new rows, the deliminator is " and "

In [10]:
keyword = " and "

In [11]:
# search and split
def search_and_split(Locations, keyword):
    results = []
    split_strings = []
    NewLocations = []
    Indx2Dup = []
    j=0
    NewLocations = Locations
    for idx, string in enumerate(Locations):
        # print(idx)
        start_idx = string.find(keyword)
        if start_idx != -1:  # Keyword found
            end_idx = start_idx + len(keyword) - 1
            results.append((idx, start_idx, end_idx))

            # Split the string into two parts: before and after the keyword
            before_keyword = string[:start_idx].strip()
            after_keyword = string[end_idx + 1:].strip()
            split_strings.append((before_keyword, after_keyword))
            
            # Append and replace new split string into original list
            # Conditionals are to look for blank spaces or punctions and not include them 
            threshold = 2
            if len(before_keyword) > threshold and len(after_keyword) > threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = before_keyword
                NewLocations.insert(idx+1, after_keyword)
                #idx is no longer real as New List grows, need to keep track
                Indx2Dup.append(idx-j)
                j+=1
                #When duplicating the rows if there are consecutive number dup from 
                    # row - number of consecutive numbers
                
            if len(before_keyword) <= threshold and len(after_keyword) > threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = after_keyword
                
            if len(before_keyword) > threshold and len(after_keyword) <= threshold:
                # print(len(NewLocations))
                # print(idx)
                NewLocations[idx] = before_keyword            

                
    return results, split_strings, NewLocations, Indx2Dup


In [12]:
# Duplicate: 
def duplicate_values_with_repeated_indices(indices, source_list):
    # Create a new list to store the results
    result = []

    for i in range(len(source_list)):
        count = indices.count(i)  # Count occurrences of the index in the indices array
        result.extend([source_list[i]] * (count + 1))  # Original + count of duplicates

    return result

In [13]:
# Testing new search and split function
# strings = [
#     "This is a sample string and it contains and perhaps denver and.",
#     "Another example without the keyword.",
#     "and is a conjunction.",
#     "Nothing here or here",
#     "But maybe here and there"
# ]

# # Search for the keyword "and" and split strings
# keyword = "and" # and is in Netherlands hahaha
# matches, splits, nLocations, Indx = search_and_split(strings, keyword)

# # Print results
# print("Keyword Matches (Start and End Indices):")
# for match in matches:
#     print(f"String index: {match[0]}, Start index: {match[1]}, End index: {match[2]}")

# print("\nSplit Strings:")
# for idx, (before, after) in enumerate(splits):
#     print(f"Original String Index: {matches[idx][0]}")
#     print(f"Before: '{before}'")
#     print(f"After: '{after}'")
    
# # print("\nResult")
# # print(nLocations)

print("\nLocation List:")
print(data_SU["Location"])
print(len(data_SU["Location"]))

matches, splits, nLocations, Indx = search_and_split(data_SU["Location"], keyword)

print("\nNew Location List:")
print(nLocations)
print(len(nLocations))

print("\nIndex to Duplicate")
print(Indx)

time.sleep(1)
clear_output()

In [14]:
# print(data_SU.keys())

In [15]:
# # Example usage
# indices = [1, 1, 3, 3, 3, 5]  # Indices for duplication
# source_list = ["a", "b", "c", "d", "e", "f"]

# # Duplicate values based on repeated indices
# result = duplicate_values_with_repeated_indices(indices, source_list)
# print(result)

In [16]:
total_unique_companies = len(data_SU['Company Name'])+len(data_LC['Company Name'])+len(data_F['Company Name'])+len(data_DC['Company Name'])

In [17]:
# Duplicate values at the specified indices for start up tab
data_SU['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_SU['Company Name'])
data_SU['Size'] = duplicate_values_with_repeated_indices(Indx, data_SU['Size'])
data_SU['Focus'] = duplicate_values_with_repeated_indices(Indx, data_SU['Focus'])
data_SU['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_SU['CEO/Founders/Affliation'])


In [18]:
# Same for LC, F, and DC
matches, splits, nLocations, Indx = search_and_split(data_LC["Location"], keyword)
data_LC['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_LC['Company Name'])
data_LC['Size'] = duplicate_values_with_repeated_indices(Indx, data_LC['Size'])
data_LC['Focus'] = duplicate_values_with_repeated_indices(Indx, data_LC['Focus'])
data_LC['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_LC['CEO/Founders/Affliation'])

matches, splits, nLocations, Indx = search_and_split(data_F["Location"], keyword)
data_F['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_F['Company Name'])
data_F['Size'] = duplicate_values_with_repeated_indices(Indx, data_F['Size'])
data_F['Focus'] = duplicate_values_with_repeated_indices(Indx, data_F['Focus'])
data_F['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_F['CEO/Founders/Affliation'])

matches, splits, nLocations, Indx = search_and_split(data_DC["Location"], keyword)
data_DC['Company Name'] = duplicate_values_with_repeated_indices(Indx, data_DC['Company Name'])
data_DC['Size'] = duplicate_values_with_repeated_indices(Indx, data_DC['Size'])
data_DC['Focus'] = duplicate_values_with_repeated_indices(Indx, data_DC['Focus'])
data_DC['CEO/Founders/Affliation'] = duplicate_values_with_repeated_indices(Indx, data_DC['CEO/Founders/Affliation'])


In [19]:
total_unique_offices = len(data_SU['Company Name'])+len(data_LC['Company Name'])+len(data_F['Company Name'])+len(data_DC['Company Name'])

### Now I have corrected list for companies with multiple location, we convert rough individual locations into coordinates and make a new column

In [20]:
# create cache


In [21]:
# # Convert Rough Location to longitude and latitude
# # Original function without cache


# def get_coordinates_with_duplicates(locations, w_Debug=False):
#     # Initialize the geolocator
#     geolocator = Nominatim(user_agent="location_to_coordinates")
#     cache = {}  # Cache to store already-queried locations
#     results = []  # Final list of results
#     slow_down = 0.5
#     for location in locations:
#         tries = 3
#         time.sleep(slow_down)
#         if location in cache:
#             # Use cached result if available
#             results.append(cache[location])
#         else:
#             try:
#                 # Geocode the location
#                 if w_Debug == True:
#                     print(location)
#                 for i in range(tries):
#                     try:
#                         loc = geolocator.geocode(location)
#                     except GeocoderTimedOut as e:
#                         if i < tries - 1: # i is zero indexed
#                             time.sleep(1)
#                             print(Location, "failed, trying again")
#                             continue
#                         else:
#                             raise
#                 if loc:
#                     coordinates = (loc.latitude, loc.longitude)
#                 else:
#                     coordinates = None
#                     print('Could find coordinates for ',location, '.')
#                     print('Please change the location name within the database.\n')
#                 # Cache the result and append to results
#                 cache[location] = coordinates
#                 results.append(coordinates)
#             except GeocoderTimedOut:
#                 # Handle timeout gracefully
#                 print("Failed at providing ", location, " at least ", tries, " number of times.")
#                 results.append(None)

#     return results

In [22]:
# Define the cache file
CACHE_FILE = "location_cache.csv"

def initialize_cache():
    """Ensure the cache file exists with the proper headers."""
    if not os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Location", "Latitude", "Longitude"])

In [23]:
def read_cache():
    """Read the cache into a dictionary."""
    initialize_cache()
    cache = {}
    with open(CACHE_FILE, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            location = row["Location"]
            latitude = float(row["Latitude"]) if row["Latitude"] else None
            longitude = float(row["Longitude"]) if row["Longitude"] else None
            cache[location] = (latitude, longitude)
    return cache

In [24]:
def write_to_cache(location, coordinates):
    """Append a new location and its coordinates to the cache."""
    with open(CACHE_FILE, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        latitude, longitude = coordinates if coordinates else (None, None)
        writer.writerow([location, latitude, longitude])


In [25]:
def is_over_water(latitude, longitude):
    """
    Check if a coordinate is over water using global_land_mask
    Link complained that there was a company located in the hudson, so this is 
    my attempt at a  quick fix
    """
    location = globe.is_land(latitude, longitude)
    if location != True:
        # print("Found water!")
        return True
    return False

In [26]:
# Convert Rough Location to longitude and latitude
def get_coordinates_with_duplicates(locations, w_Debug=False):
    geolocator = Nominatim(user_agent="location_to_coordinates")
    cache = read_cache()  # Load the cache from the CSV
    results = []  # Final list of results
    slow_down = 0.5

    for location in locations:
        tries = 3
        if location in cache:
            # Use cached result if available
            results.append(cache[location])
            print(location, 'coords pulled from cache.')
            time.sleep(0.1)
        else:
            try:
                if w_Debug:
                    print(f"Querying for: {location}")
                coordinates = None
                for i in range(tries):
                    try:
                        time.sleep(slow_down)
                        loc = geolocator.geocode(location)
                        if loc:
                            coordinates = (loc.latitude, loc.longitude)
                        break  # Break if successful
                    except GeocoderTimedOut:
                        if i < tries - 1:
                            time.sleep(1)
                            print(f"Retrying {location} ({i+1}/{tries})...")
                        else:
                            print(f"Failed to geocode {location} after {tries} tries.")
                            coordinates = None

                if not coordinates:
                    print(f"Could not find coordinates for {location}.")
                    print("Please check or modify the location name in your database.\n")

                # Cache the result
                cache[location] = coordinates
                write_to_cache(location, coordinates)
                results.append(coordinates)
            except Exception as e:
                print(f"Error processing {location}: {e}")
                results.append(None)
    time.sleep(1)
    return results


In [27]:

# if __name__ == "__main__":
#     locations = ["New York", "Los Angeles", "Chicago", "Nonexistent Place"]
#     coordinates = get_coordinates_with_duplicates(locations, w_Debug=True)
#     print("Results:", coordinates)

In [28]:
# print(data_SU["Location"])

### Check here if any rough addresses aren't able to be converted

In [29]:
''' Get coordinates from location list Sometimes fails, change debug to true and rerun
 Could be just timing out, could be a bad location in the spread sheet 
 if timing out, you can add more time out to slow down within the function
 Added cache such that regenerating map can be done very quickly. If cache is empty
 finding all coords will take a couple mins
'''

' Get coordinates from location list Sometimes fails, change debug to true and rerun\n Could be just timing out, could be a bad location in the spread sheet \n if timing out, you can add more time out to slow down within the function\n Added cache such that regenerating map can be done very quickly. If cache is empty\n finding all coords will take a couple mins\n'

In [30]:
debug = True
clear_o = True
coordinates_SU = get_coordinates_with_duplicates(data_SU["Location"], debug)
if clear_o == True: 
    clear_output()

In [31]:
coordinates_LC = get_coordinates_with_duplicates(data_LC["Location"], debug)
if clear_o == True: 
    clear_output()

In [32]:
coordinates_F = get_coordinates_with_duplicates(data_F["Location"], debug)
if clear_o == True: 
    clear_output()

In [33]:
coordinates_DC = get_coordinates_with_duplicates(data_DC["Location"], debug)
if clear_o == True: 
    clear_output()

In [34]:
# # Example list of locations
# locations = [
#     "Boston, MA, USA",
#     "New York, NY, USA",
#     "Boston, MA, USA",  # Duplicate
#     "San Francisco, CA, USA",
#     "New York, NY, USA"  # Duplicate
# ]

# # Get coordinates
# coordinates = get_coordinates_with_duplicates(locations)

# # Print results
# for location, coord in zip(locations, coordinates):
#     if coord:
#         print(f"{location}: Latitude = {coord[0]}, Longitude = {coord[1]}")
#     else:
#         print(f"{location}: Coordinates not found.")

### Cost-of-living 

In [35]:
# Maybe scrape from Numbeo
# Kinda difficult to find an easy dataset for this 

### Plot coordinates on map

In [36]:
# https://python-visualization.github.io/folium/latest/index.html

In [37]:
def generate_random_pair():
    pairs = []
    Variation_Magnitiude = 0.02
    Exclude_Zone = 0.005
    while True:
        num1 = random.uniform(-Variation_Magnitiude, Variation_Magnitiude)
        num2 = random.uniform(-Variation_Magnitiude, Variation_Magnitiude)
        if not (-Exclude_Zone <= num1 <= Exclude_Zone or -Exclude_Zone <= num2 <= Exclude_Zone):
            pairs = [num1, num2]
            break
    return pairs

# # Example usage
# random_pair = generate_random_pair()
# print(f"Generated random pair: {random_pair}")

In [38]:
# # Example using folium 
# m = folium.Map(location=[41,29])

# # folium.Circle(
# #     radius=5000,
# #     location=[41,29],
# #     color='crimson',
# #     fill=False,).add_to(m)

# # folium.CircleMarker(location=(41,29),radius=100, fill_color='red').add_to(m)

# folium.Marker(location=[41,29], popup =  'Sakarya').add_to(m)
# new_pair = generate_random_pair()
# folium.Marker(location=[41+new_pair[0],29+new_pair[1]], popup =  'Sakarya').add_to(m)

# minimap = plugins.MiniMap()
# m.add_child(minimap)

# m


In [39]:
# Plotting SUs on World Map
# Add a slight random variation to coordinates to make markers in high density areas (the bay) more legible 

#Init map with starting location at SF
SF = [37.7749, -122.4194]
USA = [40, -98]
Zoom = 4
CompanyMap = folium.Map(USA, tiles="cartodb positron", zoom_start=Zoom)
color_SU = "darkblue"
color_LC = "red"
color_F = "green"
color_DC = "purple"
popup_width=300
show_SU = True
show_LC = True
show_F = True
show_DC = True

#Adds time to generate map, but if so desired this function will ensure markers are not over water. 
check_for_water = False

if show_SU == True:
    for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
        in  zip(coordinates_SU, data_SU['Company Name'], data_SU['Size'], \
                data_SU['Focus'],data_SU['CEO/Founders/Affliation']):  
        variation = generate_random_pair()
        text = new_CompName+'<br>'+'Size: ' + str(new_Size) +'<br>'+ \
                     'Focus: ' + new_Focus +'<br>'+ 'Key Members and Affliations: ' \
                     + new_Founders
        iframe = folium.IFrame(text)
        popup = folium.Popup(iframe, min_width=popup_width,max_width=popup_width)
        if check_for_water == True: 
            while True:
                    variation = generate_random_pair()
                    mark_loc = [a + b for a, b in zip(new_cords, variation)]
                    if not is_over_water(mark_loc[0],mark_loc[1]):
                        break
        else: 
            mark_loc = [a + b for a, b in zip(new_cords, variation)]
            
        marker = folium.Marker(location = mark_loc, 
                      tooltip = new_CompName,
                      popup = popup,
                     icon = folium.Icon(icon="info-sign", icon_color ="white", \
                                      color = color_SU)).add_to(CompanyMap)

if show_LC == True:
    for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
        in  zip(coordinates_LC, data_LC['Company Name'], data_LC['Size'], \
                data_LC['Focus'],data_LC['CEO/Founders/Affliation']):  
        variation = generate_random_pair()
        text = new_CompName+'<br>'+'Size: ' + str(new_Size) +'<br>'+ \
                     'Focus: ' + new_Focus +'<br>'+ 'Key Members and Affliations: ' \
                     + new_Founders
        iframe = folium.IFrame(text)
        popup = folium.Popup(iframe, min_width=popup_width,max_width=popup_width)
        if check_for_water == True: 
            while True:
                    variation = generate_random_pair()
                    mark_loc = [a + b for a, b in zip(new_cords, variation)]
                    if not is_over_water(mark_loc[0],mark_loc[1]):
                        break
        else: 
            mark_loc = [a + b for a, b in zip(new_cords, variation)]
        maker = folium.Marker(location = mark_loc, \
                      tooltip = new_CompName,
                      popup = popup,
                     icon=folium.Icon(icon="info-sign", icon_color ="white", color = color_LC)).add_to(CompanyMap)

if show_F == True:
    for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
        in  zip(coordinates_F, data_F['Company Name'], data_F['Size'], \
                data_F['Focus'],data_F['CEO/Founders/Affliation']): 
        variation = generate_random_pair()
        text = new_CompName+'<br>'+'Size: ' + str(new_Size) +'<br>'+ \
                     'Focus: ' + new_Focus +'<br>'+ 'Key Members and Affliations: ' \
                     + new_Founders
        iframe = folium.IFrame(text)
        popup = folium.Popup(iframe, min_width=popup_width,max_width=popup_width)
        if check_for_water == True: 
            while True:
                    variation = generate_random_pair()
                    mark_loc = [a + b for a, b in zip(new_cords, variation)]
                    if not is_over_water(mark_loc[0],mark_loc[1]):
                        break
        else: 
            mark_loc = [a + b for a, b in zip(new_cords, variation)]
        maker = folium.Marker(location = mark_loc, 
                      tooltip = new_CompName,
                      popup = popup,
                     icon = folium.Icon(icon="info-sign", icon_color ="white", color = color_F)).add_to(CompanyMap)

if show_DC == True:
    for new_cords, new_CompName, new_Size, new_Focus, new_Founders \
        in  zip(coordinates_DC, data_DC['Company Name'], data_DC['Size'], \
                data_DC['Focus'],data_DC['CEO/Founders/Affliation']):  
        variation = generate_random_pair()
        text = new_CompName+'<br>'+'Size: ' + str(new_Size) +'<br>'+ \
                     'Focus: ' + new_Focus +'<br>'+ 'Key Members and Affliations: ' \
                     + new_Founders
        iframe = folium.IFrame(text)
        popup = folium.Popup(iframe, min_width=popup_width,max_width=popup_width)
        if check_for_water == True: 
            while True:
                    variation = generate_random_pair()
                    mark_loc = [a + b for a, b in zip(new_cords, variation)]
                    if not is_over_water(mark_loc[0],mark_loc[1]):
                        break
        else: 
            mark_loc = [a + b for a, b in zip(new_cords, variation)]
        maker = folium.Marker(location = mark_loc, 
                      tooltip = new_CompName,
                      popup = popup, 
                     icon = folium.Icon(icon="info-sign", icon_color ="white", color = color_DC)).add_to(CompanyMap)

#Adding Minimap    
minimap = plugins.MiniMap()
CompanyMap.add_child(minimap)

# Define the legend
if show_SU or show_DC or show_F or show_LC == True:
    legend_html = '''
    <div style="position: fixed; 
         bottom: 20px; left: 20px; width: 160px; height: '''
    legend_html = legend_html + str((show_SU + show_DC + show_F + show_LC)*25 +10) + '''px; 
         border:2px solid grey; z-index:9999; font-size:14px;
         background-color:white; opacity: 0.85;">
         &nbsp; <b>Company Type</b> <br> ''' 
    if show_SU == True:
        legend_html = legend_html+'''
         &nbsp; <i class="fa fa-circle" style="color:''' + color_SU + '''"></i>  Small / Start Up &nbsp; <br>'''
    if show_LC == True:
        legend_html = legend_html+'''
         &nbsp; <i class="fa fa-circle" style="color:''' + color_LC + '''"></i>  Large &nbsp; <br>'''
    if show_F == True:
        legend_html = legend_html+'''
         &nbsp; <i class="fa fa-circle" style="color:''' + color_F + '''"></i>  Foundry &nbsp; <br>'''
    if show_DC == True:
        legend_html = legend_html+'''
         &nbsp; <i class="fa fa-circle" style="color:''' + color_DC + '''"></i>  Defense Contractor &nbsp; <br>'''
    legend_html = legend_html +'''       
    </div>
    '''
    
# Add the legend to the map
CompanyMap.get_root().html.add_child(folium.Element(legend_html))

#Display Map
CompanyMap

# ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
#  |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
#  |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
#  |           'gray', 'black', 'lightgray']

In [40]:
isSaving = True
if isSaving:
    CompanyMap.save("CurrentCompanyMap.html")

In [41]:
# help(folium.Map)

In [42]:
# How many company locations are being shown?
location_list = []
with open(CACHE_FILE, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        location_list.append(row)
print("Number of companies represented: ", total_unique_companies)
print("Total number of offices: ", total_unique_offices)
# print("Number of unique locations:", len(location_list))


Number of companies represented:  109
Total number of offices:  218
